In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import pyedflib

import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import ipywidgets as widgets
from matplotlib import gridspec
import matplotlib.image as mpimg
from scipy.signal import hilbert, filtfilt, find_peaks, lfilter

project_root = '..'
sys.path.append(project_root)

from sleeprnn.data.inta_ss import IntaSS, NAMES, IDS_TEST
from sleeprnn.data import utils, stamp_correction
from sleeprnn.detection import metrics
from sleeprnn.data.loader import RefactorUnpickler
from sleeprnn.common import constants, pkeys

SEED_LIST = [123, 234, 345, 456]
RESULTS_PATH = os.path.join(project_root, 'results')
COMPARISON_PATH = os.path.join(project_root, 'resources', 'comparison_data')
DPI = 100
CUSTOM_COLOR = {'red': '#c62828', 'grey': '#455a64', 'blue': '#0277bd', 'green': '#43a047'} 

%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:

# look for intersections between stamps of the same validity
def overlap_matrix(events, detections):
    # Matrix of overlap, rows are events, columns are detections
    n_det = detections.shape[0]
    n_gs = events.shape[0]
    overlaps = np.zeros((n_gs, n_det))
    for i in range(n_gs):
        candidates = np.where(
            (detections[:, 0] <= events[i, 1])
            & (detections[:, 1] >= events[i, 0]))[0]
        for j in candidates:
            intersection = min(
                events[i, 1], detections[j, 1]
            ) - max(
                events[i, 0], detections[j, 0]
            ) + 1
            if intersection > 0:
                overlaps[i, j] = 1
    return overlaps

def overlapping_groups(overlap_matrix):
    groups_overlap = [[0]]
    for i in range(overlap_matrix.shape[0]):
        visited = np.any([i in single_group for single_group in groups_overlap])
        if not visited:
            # Check if intersects with an existent group
            added = False
            for single_group in groups_overlap:
                is_overlapping = np.any(overlap_matrix[i, single_group])
                if is_overlapping:
                    single_group.append(i)
                    added = True
            if not added:
                # Then variable is a new group
                groups_overlap.append([i])
    return groups_overlap

In [ ]:
# Load dataset
dataset = IntaSS(load_checkpoint=True, params={pkeys.NORM_COMPUTATION_MODE: constants.NORM_GLOBAL})
dataset_name = constants.INTA_SS_NAME

# Load predictions
ckpt_folder = os.path.join('20190706_inta_05_n2_train_inta_ss', 'v15')
optimal_thr_list = [0.42, 0.44, 0.48, 0.5]
task_mode = constants.N2_RECORD
seed_id_list = [0, 1, 2, 3]
n_seeds = len(seed_id_list)
set_list = [constants.TRAIN_SUBSET, constants.VAL_SUBSET, constants.TEST_SUBSET]
which_expert = 1
verbose = False
predictions_dict = {}
for k in seed_id_list:
    # Restore predictions
    ckpt_path = os.path.abspath(os.path.join(
        RESULTS_PATH,
        'predictions_%s' % dataset_name,
        ckpt_folder,
        'seed%d' % k
    ))
    this_dict = {}
    for set_name in set_list:
        filename = os.path.join(
                ckpt_path,
                'prediction_%s_%s.pkl' % (task_mode, set_name))
        with open(filename, 'rb') as handle:
            this_pred = RefactorUnpickler(handle).load()
        this_dict[set_name] = this_pred
    predictions_dict[k] = this_dict
    print('Loaded seed %d/%d from %s' % (k + 1, n_seeds, ckpt_path))
print('Optimal thr:', optimal_thr_list)

In [ ]:
# Load stamps of subject
subject_id = 11  # 4, 11

print('Loading S%02d' % subject_id)
fs = dataset.fs
path_dataset = dataset.dataset_dir
path_stamps = os.path.join(path_dataset, 'label/spindle', 'SS_%s.txt' % NAMES[subject_id - 1])
path_signals = os.path.join(path_dataset, 'register', '%s.rec' % NAMES[subject_id - 1]) 

with pyedflib.EdfReader(path_signals) as file:
    signal = file.readSignal(0)
    channel_name = file.getLabel(0)
    print('Reading', channel_name)
    signal_len = signal.shape[0]
data = np.loadtxt(path_stamps)
for_this_channel = (data[:, -1] == 1)
data = data[for_this_channel]
data = np.round(data).astype(np.int32)

# Remove zero duration marks, and ensure that start time < end time
new_data = []
for i in range(data.shape[0]):
    if data[i, 0] > data[i, 1]:
        print('End time < Start time fixed')
        aux = data[i, 0]
        data[i, 0] = data[i, 1]
        data[i, 1] = aux
        new_data.append(data[i, :])
    elif data[i, 0] < data[i, 1]:
        new_data.append(data[i, :])
    else:  # Zero duration (equality)
        print('Zero duration stamp found and removed')
data = np.stack(new_data, axis=0)

# Remove stamps outside signal boundaries
new_data = []
for i in range(data.shape[0]):
    if data[i, 1] < signal_len:
        new_data.append(data[i, :])
    else:
        print('Stamp outside boundaries found and removed')
data = np.stack(new_data, axis=0)

raw_stamps = data[:, [0, 1]]
valid = data[:, 4]
raw_stamps_0 = raw_stamps[valid == 0]
raw_stamps_1 = raw_stamps[valid == 1]
raw_stamps_2 = raw_stamps[valid == 2]
print(
    'Valid 0', raw_stamps_0.shape, 'Min dur [s]', (raw_stamps_0[:, 1] - raw_stamps_0[:, 0]).min()/fs, 
    'Max dur [s]', (raw_stamps_0[:, 1] - raw_stamps_0[:, 0]).max()/fs)
print(
    'Valid 1', raw_stamps_1.shape, 'Min dur [s]', (raw_stamps_1[:, 1] - raw_stamps_1[:, 0]).min()/fs, 
    'Max dur [s]', (raw_stamps_1[:, 1] - raw_stamps_1[:, 0]).max()/fs)
print(
    'Valid 2', raw_stamps_2.shape, 'Min dur [s]', (raw_stamps_2[:, 1] - raw_stamps_2[:, 0]).min()/fs, 
    'Max dur [s]', (raw_stamps_2[:, 1] - raw_stamps_2[:, 0]).max()/fs)

del raw_stamps_0

overlap_m = overlap_matrix(raw_stamps_1, raw_stamps_1)
groups_overlap_1 = overlapping_groups(overlap_m)

n_overlaps_1 = [len(single_group) for single_group in groups_overlap_1]
values_1, counts_1 = np.unique(n_overlaps_1, return_counts=True)
print('\nSize of overlapping groups for Valid 1')
for value, count in zip(values_1, counts_1):
    print('%d marks: %d times' % (value, count))

overlap_m = overlap_matrix(raw_stamps_2, raw_stamps_2)
groups_overlap_2 = overlapping_groups(overlap_m)
n_overlaps_2 = [len(single_group) for single_group in groups_overlap_2]
values_2, counts_2 = np.unique(n_overlaps_2, return_counts=True)
print('\nSize of overlapping groups for Valid 2')
for value, count in zip(values_2, counts_2):
    print('%d marks: %d times' % (value, count))

max_overlaps_1 = values_1.max() - 1
max_overlaps_2 = values_2.max() - 1
max_overlaps = np.max([max_overlaps_1, max_overlaps_2])

In [ ]:
# Signal
signal_dict = {}
fs_dict = {}
with pyedflib.EdfReader(path_signals) as file:
    signal_names = file.getSignalLabels()
    
print(signal_names)
eeg_names = signal_names[:5]
other_names = signal_names[5:6]  + signal_names[7:8]
to_show_names = eeg_names + other_names
print(to_show_names)  

with pyedflib.EdfReader(path_signals) as file:   
    for k, name in enumerate(signal_names):
        if name in to_show_names:
            this_signal = file.readSignal(k)
            fs_dict[name] = file.getSampleFrequency(k)
            signal_dict[name] = this_signal

In [ ]:
# -*- coding: utf-8 -*-
# -*- mode: python -*-
# Adapted from mpl_toolkits.axes_grid1
# LICENSE: Python Software Foundation (http://docs.python.org/license.html)

from matplotlib.offsetbox import AnchoredOffsetbox
class AnchoredScaleBar(AnchoredOffsetbox):
    def __init__(self, transform, sizex=0, sizey=0, labelx=None, labely=None, loc=4,
                 pad=0.1, borderpad=0.1, sep=2, prop=None, barcolor="black", barwidth=None, 
                 **kwargs):
        """
        Draw a horizontal and/or vertical  bar with the size in data coordinate
        of the give axes. A label will be drawn underneath (center-aligned).
        - transform : the coordinate frame (typically axes.transData)
        - sizex,sizey : width of x,y bar, in data units. 0 to omit
        - labelx,labely : labels for x,y bars; None to omit
        - loc : position in containing axes
        - pad, borderpad : padding, in fraction of the legend font size (or prop)
        - sep : separation between labels and bars in points.
        - **kwargs : additional arguments passed to base class constructor
        """
        from matplotlib.patches import Rectangle
        from matplotlib.offsetbox import AuxTransformBox, VPacker, HPacker, TextArea, DrawingArea
        bars = AuxTransformBox(transform)
        if sizex:
            bars.add_artist(Rectangle((0,0), sizex, 0, ec=barcolor, lw=barwidth, fc="none"))
        if sizey:
            bars.add_artist(Rectangle((0,0), 0, sizey, ec=barcolor, lw=barwidth, fc="none"))

        if sizex and labelx:
            self.xlabel = TextArea(labelx, minimumdescent=False)
            bars = VPacker(children=[bars, self.xlabel], align="center", pad=0, sep=sep)
        if sizey and labely:
            self.ylabel = TextArea(labely)
            bars = HPacker(children=[self.ylabel, bars], align="center", pad=0, sep=sep)

        AnchoredOffsetbox.__init__(self, loc, pad=pad, borderpad=borderpad,
                                   child=bars, prop=prop, frameon=False, **kwargs)

        
def add_scalebar(ax, matchx=True, matchy=True, hidex=True, hidey=True, **kwargs):
    """ Add scalebars to axes
    Adds a set of scale bars to *ax*, matching the size to the ticks of the plot
    and optionally hiding the x and y axes
    - ax : the axis to attach ticks to
    - matchx,matchy : if True, set size of scale bars to spacing between ticks
                    if False, size should be set using sizex and sizey params
    - hidex,hidey : if True, hide x-axis and y-axis of parent
    - **kwargs : additional arguments passed to AnchoredScaleBars
    Returns created scalebar object
    """
    def f(axis):
        l = axis.get_majorticklocs()
        return len(l)>1 and (l[1] - l[0])
    
    if matchx:
        kwargs['sizex'] = f(ax.xaxis)
        kwargs['labelx'] = str(kwargs['sizex'])
    if matchy:
        kwargs['sizey'] = f(ax.yaxis)
        kwargs['labely'] = str(kwargs['sizey'])
        
    sb = AnchoredScaleBar(ax.transData, **kwargs)
    ax.add_artist(sb)

    if hidex : ax.xaxis.set_visible(False)
    if hidey : ax.yaxis.set_visible(False)
    if hidex and hidey: ax.set_frame_on(False)

    return sb

In [ ]:
subject_seed_set_dict = {
    1: (2, constants.VAL_SUBSET),
    2: (1, constants.VAL_SUBSET),
    4: (0, constants.VAL_SUBSET),
    5: (0, constants.TEST_SUBSET),
    6: (1, constants.VAL_SUBSET),
    7: (0, constants.TEST_SUBSET),
    8: (3, constants.VAL_SUBSET),
    9: (0, constants.TEST_SUBSET),
    10: (2, constants.VAL_SUBSET),
    11: (0, constants.VAL_SUBSET),
}

this_pages = dataset.get_subject_pages(subject_id=subject_id) 
this_stamps = dataset.get_subject_stamps(subject_id=subject_id)

# Power
# Sigma relative amplitude
low_sigma_freq = 10
high_sigma_freq = 16
window_size = 101
# signal_to_power = utils.broad_filter(signal_dict['F4-C4'], fs, lowcut=0.1, highcut=30)
signal_original = dataset.get_subject_signal(subject_id=subject_id, normalize_clip=False)
signal_to_power, _ = utils.norm_clip_signal_global(signal_original, dataset.global_std, clip_value=3)
print(signal_to_power.min(), signal_to_power.max())
signal_sigma = utils.filter_windowed_sinusoidal(signal_to_power, fs, 13, 41)

kernel = np.ones(window_size) / window_size
rms_broad = lfilter(kernel, 1.0, signal_to_power ** 2)
n_shift = (window_size-1) // 2
instantaneous_rms_broad = np.zeros(rms_broad.shape)
instantaneous_rms_broad[:-n_shift] = rms_broad[n_shift:]

kernel = np.ones(window_size) / window_size
rms_narrow = lfilter(kernel, 1.0, signal_sigma ** 2)
n_shift = (window_size-1) // 2
instantaneous_rms_narrow = np.zeros(rms_narrow.shape)
instantaneous_rms_narrow[:-n_shift] = rms_narrow[n_shift:]

absolute_power = instantaneous_rms_narrow
relative_power = instantaneous_rms_narrow / (instantaneous_rms_broad + 1e-3)

max_absolute_power = np.percentile(absolute_power, 99.5)
# max_absolute_power = absolute_power.max()
min_absolute_power = absolute_power.min()
max_relative_power = np.percentile(relative_power, 99.5)
# max_relative_power = relative_power.max()
min_relative_power = relative_power.min()

absolute_power = (absolute_power-min_absolute_power) / (max_absolute_power-min_absolute_power)
relative_power = (relative_power-min_relative_power) / (max_relative_power-min_relative_power)

# Peak density
window_size = 101
signal_for_peaks = utils.broad_filter(signal_dict['F4-C4'], fs, lowcut=0.1, highcut=20)
segment_peaks, _ = find_peaks(signal_for_peaks)
peaks_onehot = np.zeros(signal_for_peaks.shape)
peaks_onehot[segment_peaks] = 1
kernel = np.ones(window_size)
peak_density = lfilter(kernel, 1.0, peaks_onehot) / (window_size / fs)
n_shift = (window_size-1) // 2
instantaneous_frequency = np.zeros(peak_density.shape)
instantaneous_frequency[:-n_shift] = peak_density[n_shift:]
kernel = np.hanning(21)
kernel = kernel / kernel.sum()
instantaneous_frequency = lfilter(kernel, 1.0, instantaneous_frequency)
n_shift = (kernel.size-1) // 2
tmp = instantaneous_frequency
instantaneous_frequency_complete = np.zeros(tmp.shape)
instantaneous_frequency_complete[:-n_shift] = tmp[n_shift:]

abs_tol = 0.1
rel_tol = 0.15
low_sigma_freq = 10
high_sigma_freq = 16
valid_abs = (absolute_power >= abs_tol).astype(np.int32)
valid_rel = (relative_power >= rel_tol).astype(np.int32)
valid_freq = (instantaneous_frequency_complete >= low_sigma_freq).astype(np.int32) * (instantaneous_frequency_complete <= high_sigma_freq).astype(np.int32)
valid_both = valid_abs * valid_rel * valid_freq
primitive_mark = utils.seq2stamp(valid_both)

print_idx = np.where((primitive_mark[:, 0]>=1660000) & (primitive_mark[:, 0]<=1664000))[0]
print_stamps = primitive_mark[print_idx, :]
print('First pass (power and freq thresholding)')
print(print_stamps/fs)

primitive_mark = stamp_correction.combine_close_stamps(
    primitive_mark, fs, 0.1)
primitive_mark = stamp_correction.filter_duration_stamps(
    primitive_mark, fs, 0.1, 5)
primitive_mark = stamp_correction.combine_close_stamps(
    primitive_mark, fs, 0.3)

print_idx = np.where((primitive_mark[:, 0]>=1660000) & (primitive_mark[:, 0]<=1664000))[0]
print_stamps = primitive_mark[print_idx, :]
print('Second pass (duration control sep 0.1, dur [0.1 5], then sep 0.3)')
print(print_stamps/fs)

# Now remove low power marks
mean_power_abs_tol = 0.15
mean_power_rel_tol = 0.2
improved_primitive_mark = []
abs_power_list = []
rel_power_list = []
for single_primitive in primitive_mark:
    stamp_abs_power = np.mean(absolute_power[single_primitive[0]:single_primitive[1]])
    stamp_rel_power = np.mean(relative_power[single_primitive[0]:single_primitive[1]])
    abs_power_list.append(stamp_abs_power)
    rel_power_list.append(stamp_rel_power)
    if (stamp_abs_power >= mean_power_abs_tol) and (stamp_rel_power >= mean_power_rel_tol):
        improved_primitive_mark.append(single_primitive)
primitive_mark = np.stack(improved_primitive_mark, axis=0)

print_idx = np.where((primitive_mark[:, 0]>=1660000) & (primitive_mark[:, 0]<=1664000))[0]
print_stamps = primitive_mark[print_idx, :]
print('Third pass (mean power thresholding)')
print(print_stamps/fs)

primitive_mark = stamp_correction.combine_close_stamps(
    primitive_mark, fs, 0.8)
primitive_mark = stamp_correction.filter_duration_stamps(
    primitive_mark, fs, 0.3, 3)

print_idx = np.where((primitive_mark[:, 0]>=1660000) & (primitive_mark[:, 0]<=1664000))[0]
print_stamps = primitive_mark[print_idx, :]
print('Fourth pass (duration control sep 0.8, dur [0.3 3])')
print(print_stamps/fs)

plt.figure(figsize=(8, 3))
plt.title('Mean absolute power inside primitive')
plt.hist(abs_power_list, bins=100)
#plt.xlim([0, 1])
plt.show()

plt.figure(figsize=(8, 3))
plt.title('Mean relative power inside primitive')
plt.hist(rel_power_list, bins=100)
#plt.xlim([0, 1])
plt.show()
    
    
# Model predictions
model_stamps_list = []
iou_list = np.arange(1, 10) * 0.1
f1_vs_iou_list = []
this_seed = subject_seed_set_dict[subject_id][0]
set_name = subject_seed_set_dict[subject_id][1]
prediction_obj = predictions_dict[this_seed][set_name]
prediction_obj.set_probability_threshold(optimal_thr_list[this_seed])
this_detections = prediction_obj.get_subject_stamps(subject_id=subject_id)
model_stamps_list.append(this_detections)
f1_vs_iou = metrics.metric_vs_iou(
    this_stamps, this_detections, iou_list)
fig, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=80)
ax.plot(iou_list, f1_vs_iou)
ax.set_xlim([0.1 - 0.02, 0.9 + 0.02])
ax.set_ylim([0.1 - 0.02, 0.9 + 0.02])
ax.set_yticks([0.1*i for i in range(1, 10)])
ax.set_xticks([0.1*i for i in range(1, 10)])
ax.tick_params(labelsize=8.5)
ax.set_xlabel('IoU Threshold', fontsize=8.5)
ax.set_ylabel('F1-score', fontsize=8.5)
ax.yaxis.grid()
plt.show()

thr_size = 5
print('Overlapping groups for Valid 2 with size at least %d' % thr_size)
to_print_list = [sub_group for sub_group in groups_overlap_2 if len(sub_group)>=thr_size]
for single_group in to_print_list:
    single_page = int(raw_stamps_2[single_group[0], :].mean() / dataset.page_size)
    print('Group size %d. Page idx: %d. Group: %s' % (len(single_group), this_pages.tolist().index(single_page), single_group))

In [ ]:
model_stamps_color = CUSTOM_COLOR['blue']
expert_stamps_color = CUSTOM_COLOR['red']
stamps_color = '#B71C1C'

def filter_stamps(stamps, single_page, page_size):
    stamps_pages = (stamps / page_size).astype(np.int32)
    useful_idx = np.where((stamps_pages[:, 0] == single_page) | (stamps_pages[:, 1] == single_page))[0]
    useful_stamps = stamps[useful_idx, :]
    return useful_stamps

def plot_page(page_idx, show_model, show_expert, show_primitive):
    
    microvolt_per_second = 100  # Aspect ratio
    
    fig = plt.figure(figsize=(12, 1.2*len(to_show_names)+5), dpi=DPI)
    gs = gridspec.GridSpec(6, 1, height_ratios=[1, 1, 2, 2, 2, 4*len(to_show_names)])
    
    page_chosen = this_pages[page_idx]
    page_start = page_chosen * dataset.page_size
    page_end = page_start + dataset.page_size
    
    if show_expert:
        segment_stamps = filter_stamps(this_stamps, page_chosen, dataset.page_size)
        segment_stamps_valid_1 = filter_stamps(raw_stamps_1, page_chosen, dataset.page_size)
        segment_stamps_valid_2 = filter_stamps(raw_stamps_2, page_chosen, dataset.page_size)
    else:
        segment_stamps = []
        segment_stamps_valid_1 = []
        segment_stamps_valid_2 = []
        
    segment_model_stamps = []
    for single_stamps in model_stamps_list:
        if show_model:
            segment_single_stamps = filter_stamps(single_stamps, page_chosen, dataset.page_size)
            segment_model_stamps.append(segment_single_stamps)
        else:
            segment_model_stamps.append([])
            
    if show_primitive:
        segment_primitive = filter_stamps(primitive_mark, page_chosen, dataset.page_size)
    else:
        segment_primitive = []
    
    time_axis = np.arange(page_start, page_end) / fs
    x_ticks = time_axis[0] + np.arange(5, 21, 5)
    
    gs_idx = 0

    # Show valid 1
    delta_y = 0.1
    ax = fig.add_subplot(gs[gs_idx])

    for j, this_stamp in enumerate(segment_stamps_valid_1):
        n_stamps = len(segment_stamps_valid_1)
        dy_for_stamp = 1/n_stamps
        ax.plot(this_stamp/fs, [j*dy_for_stamp, j*dy_for_stamp], color=stamps_color, linewidth=1.5)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_ylim([-delta_y, 1+delta_y])
    ax.set_xlim([time_axis[0], time_axis[-1]])
    ax.set_ylabel('V1', fontsize=8)
    
    # Show valid 2
    delta_y = 0.1
    gs_idx = gs_idx + 1
    ax = fig.add_subplot(gs[gs_idx])
    for j, this_stamp in enumerate(segment_stamps_valid_2):
        n_stamps = len(segment_stamps_valid_2)
        dy_for_stamp = 1/n_stamps
        ax.plot(this_stamp/fs, [j*dy_for_stamp, j*dy_for_stamp], color=stamps_color, linewidth=1.5)
    ax.set_xticks(np.arange(time_axis[0], time_axis[-1], 0.5), minor=True)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_ylim([-delta_y, 1+delta_y])
    ax.set_xlim([time_axis[0], time_axis[-1]])
    ax.set_ylabel('V2', fontsize=8)   
    # ax.set_xlabel('Time [s]', fontsize=8)
    ax.tick_params(labelsize=8.5)
    
    # Show instantaneous frequency
    low_sigma_freq = 10
    high_sigma_freq = 16
    window_size = 101
    
    gs_idx = gs_idx + 1
    ax = fig.add_subplot(gs[gs_idx])
    name = 'F4-C4'
    segment_fs = fs_dict[name]
    segment_start = int(page_chosen * dataset.page_duration * segment_fs)
    segment_end = int(segment_start + dataset.page_duration * segment_fs)
    segment_signal = signal_dict[name][segment_start:segment_end]
    segment_time_axis = np.arange(segment_start, segment_end) / segment_fs
    
    # Peak density
    peak_border = int(5*fs)
    segment_signal_for_peaks = signal_dict[name][segment_start-peak_border:segment_end+peak_border]
    signal_for_peaks = np.zeros(segment_signal_for_peaks.shape)
    filtered_for_peaks = utils.broad_filter(segment_signal_for_peaks, fs, lowcut=0.1, highcut=20)
    signal_for_peaks[fs:-fs] = filtered_for_peaks[fs:-fs]
    segment_peaks, _ = find_peaks(signal_for_peaks)
    peaks_onehot = np.zeros(signal_for_peaks.shape)
    peaks_onehot[segment_peaks] = 1
    
    kernel = np.ones(window_size)
    peak_density = lfilter(kernel, 1.0, peaks_onehot) / (window_size / fs)
    n_shift = (window_size-1) // 2
    instantaneous_frequency = np.zeros(peak_density.shape)
    instantaneous_frequency[:-n_shift] = peak_density[n_shift:]
    
    kernel = np.hanning(21)
    kernel = kernel / kernel.sum()
    instantaneous_frequency = lfilter(kernel, 1.0, instantaneous_frequency)
    n_shift = (kernel.size-1) // 2
    tmp = instantaneous_frequency
    instantaneous_frequency = np.zeros(tmp.shape)
    instantaneous_frequency[:-n_shift] = tmp[n_shift:]
    
    instantaneous_frequency = instantaneous_frequency[peak_border:-peak_border]
    
    print('V1')
    for j, this_stamp in enumerate(segment_stamps_valid_1):
        start_slice = max(this_stamp[0] - segment_start, 0)
        end_slice = min(this_stamp[1] - segment_start + 1, segment_end)
        snippet_f = instantaneous_frequency[start_slice:end_slice]
        inside_samples = np.where((snippet_f<=high_sigma_freq) & (snippet_f>=low_sigma_freq))[0]
        print(
            'mean f %1.2f, dt %1.2f [s], dt inside band %1.2f' 
            % (snippet_f.mean(), snippet_f.size/fs, inside_samples.size/fs))
    print('V2')
    for j, this_stamp in enumerate(segment_stamps_valid_2):
        start_slice = max(this_stamp[0] - segment_start, 0)
        end_slice = min(this_stamp[1] - segment_start + 1, segment_end)
        snippet_f = instantaneous_frequency[start_slice:end_slice]
        inside_samples = np.where((snippet_f<=high_sigma_freq) & (snippet_f>=low_sigma_freq))[0]
        print(
            'mean f %1.2f, dt %1.2f [s], dt inside band %1.2f, %% inside band %1.2f' 
            % (snippet_f.mean(), snippet_f.size/fs, inside_samples.size/fs, 100*inside_samples.size/snippet_f.size))
    
    ax.plot(
        segment_time_axis, instantaneous_frequency, linewidth=1, color=CUSTOM_COLOR['grey'])
    ax.plot(
        segment_time_axis, instantaneous_frequency_complete[segment_start:segment_end], linewidth=1, color=CUSTOM_COLOR['red'])
    ax.plot([time_axis[0], time_axis[-1]], [low_sigma_freq, low_sigma_freq], 'k--', linewidth=1)
    ax.plot([time_axis[0], time_axis[-1]], [high_sigma_freq, high_sigma_freq], 'k--', linewidth=1)
    
    ax.set_xticks(np.arange(time_axis[0], time_axis[-1], 0.5), minor=True)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_ylim([4, 20])
    ax.set_xlim([time_axis[0], time_axis[-1]])
    ax.set_ylabel('f [Hz]', fontsize=8)   
    # ax.set_xlabel('Time [s]', fontsize=8)
    ax.tick_params(labelsize=8.5)

    gs_idx = gs_idx + 1
    ax = fig.add_subplot(gs[gs_idx])
    name = 'F4-C4'
    segment_fs = fs_dict[name]
    segment_start = int(page_chosen * dataset.page_duration * segment_fs)
    segment_end = int(segment_start + dataset.page_duration * segment_fs)
    segment_signal = signal_dict[name][segment_start:segment_end]
    segment_time_axis = np.arange(segment_start, segment_end) / segment_fs
    
    ax.plot(
        segment_time_axis, absolute_power[segment_start:segment_end], linewidth=1, color=CUSTOM_COLOR['grey'])
    ax.plot([time_axis[0], time_axis[-1]], [abs_tol, abs_tol], 'k--', linewidth=1)
    
    ax.set_xticks(np.arange(time_axis[0], time_axis[-1], 0.5), minor=True)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlim([time_axis[0], time_axis[-1]])
    ax.set_ylabel('AbsP', fontsize=8)   
    ax.set_ylim([0, 1])
    # ax.set_xlabel('Time [s]', fontsize=8)
    ax.tick_params(labelsize=8.5)
    
    gs_idx = gs_idx + 1
    ax = fig.add_subplot(gs[gs_idx])
    ax.plot(
        segment_time_axis, relative_power[segment_start:segment_end], linewidth=1, color=CUSTOM_COLOR['grey'])
    ax.plot([time_axis[0], time_axis[-1]], [rel_tol, rel_tol], 'k--', linewidth=1)
    ax.set_xticks(np.arange(time_axis[0], time_axis[-1], 0.5), minor=True)
    ax.set_xticks(x_ticks)
    ax.set_yticks([])
    ax.set_xlim([time_axis[0], time_axis[-1]])
    ax.set_ylabel('RelP', fontsize=8)   
    ax.set_xlabel('Time [s]', fontsize=8)
    ax.tick_params(labelsize=8.5)
    ax.set_ylim([0, 1])
    
    
    # Signal
    gs_idx = gs_idx + 1
    y_max = 150
    y_sep = 150
    scale = 0.5
    ax = fig.add_subplot(gs[gs_idx])
    
    for k, name in enumerate(to_show_names):
        if name == 'F4-C4':
            stamp_center = -y_sep*k
        segment_fs = fs_dict[name]
        segment_start = int(page_chosen * dataset.page_duration * segment_fs)
        segment_end = int(segment_start + dataset.page_duration * segment_fs)
        segment_signal = signal_dict[name][segment_start:segment_end]
        segment_time_axis = np.arange(segment_start, segment_end) / segment_fs
        ax.plot(
            segment_time_axis, -y_sep*k + scale*segment_signal, linewidth=1, color=CUSTOM_COLOR['grey'])
    
    add_scalebar(ax, matchx=True, matchy=True, hidex=False, hidey=False)
    model_shown = False
    expert_shown = False
    primitive_shown = False
    for single_segment_model_stamps in segment_model_stamps:
        for model_stamp in single_segment_model_stamps:
            if not model_shown:
                label = 'Model'
            else:
                label = None
            ax.fill_between(
                model_stamp / fs, 50+stamp_center, -50+stamp_center, 
                facecolor=model_stamps_color, alpha=0.5,  label=label)
            model_shown = True
    for expert_stamp in segment_stamps:
        if not expert_shown:
            label = 'Expert'
        else:
            label = None
        ax.plot(
            expert_stamp / fs, [stamp_center-50, stamp_center-50], 
            color=expert_stamps_color, linewidth=2, label=label)
        expert_shown = True
        
    for primitive_stamp in segment_primitive:
        if not primitive_shown:
            label = 'Primitive'
        else:
            label = None
        ax.plot(
            primitive_stamp / fs, [stamp_center-30, stamp_center-30], 
            color=CUSTOM_COLOR['green'], linewidth=2, label=label)
        primitive_shown = True
    
    ax.set_yticks([-y_sep*k for k in range(len(to_show_names))])
    ax.set_yticklabels(to_show_names)
    ax.set_xlim([time_axis[0], time_axis[-1]])
    ax.set_ylim([-y_max -y_sep*(len(to_show_names)-1) - 30, y_max])
    ax.set_title('Subject %d (%s INTA). Page in record: %d. (intervals of 0.5s are shown).' 
                 % (subject_id, NAMES[subject_id-1], page_chosen), fontsize=10)
    ax.set_xticks(x_ticks)
    ax.set_xticks(np.arange(time_axis[0], time_axis[-1], 0.5), minor=True)
    ax.grid(b=True, axis='x', which='minor')
    ax.tick_params(labelsize=8.5)
    ax.set_aspect(1/microvolt_per_second)
    ax.set_xlabel('Time [s]', fontsize=8)

    if model_shown or expert_shown:
        lg = ax.legend(loc='upper right', fontsize=8)
        for lh in lg.legendHandles:
            lh.set_alpha(1.0)
    
    plt.tight_layout()
    plt.show()

In [ ]:
"""
Casos

11: Peor performance

- Consultas 1 vs 2, valid 2
- Consultas delimitación de inicio y fin en mayores discordancias
- ¿Utiliza actividad en canales vecinos para apoyar decisión y existencia y delimitación?
- Tomar 0.5s como la separación mínima entre marcas o 0.3s?

[O] Mostrar sin marca
Desde índice 400 a índice 405
Group size 4. Page idx: 400. Group: [205, 206, 207, 208] * 
401: (start xq v2>v1?; final xq?)
Group size 4. Page idx: 402. Group: [217, 218, 219, 220] *  ¿Por qué no se marca el de al medio, es un artefacto?
Group size 4. Page idx: 404. Group: [226, 227, 228, 229] * (al final)

Mostrar con marca (16 ejemplos)

Group size 4. Page idx: 348. Group: [113, 114, 115, 116] * 
Group size 4. Page idx: 393. Group: [180, 181, 182, 183] * 
Group size 4. Page idx: 394. Group: [184, 185, 186, 187] * 
Group size 4. Page idx: 411. Group: [263, 264, 265, 266] * (inicio y medio)
Group size 4. Page idx: 413. Group: [275, 276, 277, 278] * (medio y final)
Group size 4. Page idx: 414. Group: [286, 287, 288, 289] * 
Group size 4. Page idx: 424. Group: [324, 325, 326, 327] *

Group size 5. Page idx: 203. Group: [21, 22, 23, 24, 25] * 
Group size 5. Page idx: 428. Group: [346, 347, 348, 349, 350] * 
Group size 5. Page idx: 438. Group: [384, 385, 386, 387, 388] *
Group size 5. Page idx: 807. Group: [799, 800, 801, 802, 803] * [O]
Group size 5. Page idx: 849. Group: [876, 877, 878, 879, 880] * 
Group size 5. Page idx: 1280. Group: [1007, 1008, 1009, 1010, 1011] * 
Group size 5. Page idx: 1305. Group: [1089, 1090, 1091, 1092, 1093] * 

Group size 6. Page idx: 596. Group: [477, 478, 479, 480, 481, 482] *
Group size 6. Page idx: 751. Group: [619, 620, 621, 622, 623, 624] *
"""

start_page = 414
# s11 393 muy rapido, 414 muy lento, 203
# s4 61 288 363 407 615-616 624 639 846 863-864 878-879 922 938 1008
show_model = False
show_expert = True
show_primitive = True

style = {'description_width': 'initial'}
layout= widgets.Layout(width='1000px')
widgets.interact(
    lambda page_idx: plot_page(page_idx, show_model, show_expert, show_primitive),
    page_idx=widgets.IntSlider(
        min=0, max=this_pages.shape[0]-1, step=1, value=start_page, 
        continuous_update=False,
        style=style,
        layout=layout
    ));

In [ ]:
# idx 393 segment
segment_stamps = filter_stamps(this_stamps, 394, dataset.page_size)
# single_stamp = segment_stamps[2, :]
# Get signal
# context = 15
# start_sample = int(single_stamp.mean() - fs*context//2)
# end_sample = int(single_stamp.mean() + fs*context//2)
stamp_signal = signal_dict['F4-C4'][start_sample:end_sample]

# Filter
stamp_signal = utils.broad_filter(stamp_signal, fs, lowcut=1, highcut=25)
stamp_signal = stamp_signal[fs:-fs]

t = np.arange(stamp_signal.size) / fs
analytic_signal = hilbert(stamp_signal)
amplitude_envelope = np.abs(analytic_signal)
instantaneous_phase = np.unwrap(np.angle(analytic_signal))
kernel = np.hanning(21)
kernel = kernel / kernel.sum()
instantaneous_phase_filt = filtfilt(kernel, 1.0, instantaneous_phase)
instantaneous_frequency = np.gradient(instantaneous_phase_filt) / (2*np.pi) * fs



# plt.figure(figsize=(8, 2))
# plt.plot(t, np.imag(analytic_signal), linewidth=1)
# plt.fill_between((single_stamp-start_sample)/fs, -50, 50, facecolor=CUSTOM_COLOR['red'], alpha=0.3)
# plt.show()

# plt.figure(figsize=(8, 2))
# plt.plot(t, np.angle(analytic_signal), linewidth=1)
# plt.show()
# print(np.angle(analytic_signal).max(), np.angle(analytic_signal).min())

fig, ax = plt.subplots(5, 1, figsize=(10, 10), sharex=True)
ax[0].plot(t, np.real(analytic_signal), linewidth=1)
ax[0].fill_between((single_stamp-start_sample)/fs, -50, 50, facecolor=CUSTOM_COLOR['red'], alpha=0.3)

ax[1].plot(t, instantaneous_frequency, linewidth=1)
ax[1].plot([t[0], t[-1]], [11, 11], 'k--')
ax[1].plot([t[0], t[-1]], [15, 15], 'k--')

segment_peaks, _ = find_peaks(stamp_signal)
peaks_onehot = np.zeros(t.shape)
peaks_onehot[segment_peaks] = 1
window_size = 50
kernel = np.kaiser(window_size, beta=1)
peak_density = lfilter(kernel, 1.0, peaks_onehot) / (window_size / fs)

ax[2].plot(t, peaks_onehot, linewidth=1)

ax[3].plot(t, peak_density, linewidth=1)
ax[3].plot([t[0], t[-1]], [11, 11], 'k--')
ax[3].plot([t[0], t[-1]], [15, 15], 'k--')

plt.show()

In [ ]:
# S11 histogram of mean frequencies of raw marks
